In [7]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [8]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available



In [9]:

if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=laptop&ref=nb_sb_noss"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)


In [10]:
amazon_df

,title,price,rating,reviews,availability
0,Lenovo IdeaPad Gaming Chromebook - 2022 - Chro...,,3.9 out of 5 stars,12 ratings,Only 8 left in stock - order soon
1,"HP 2022 New 15 Laptop, 15.6"" HD LED Display, I...",,4.3 out of 5 stars,610 ratings,Not Available
2,ASUS Vivobook Go 15 L510 Thin & Light Laptop C...,,4.3 out of 5 stars,"1,594 ratings",In Stock
3,"SGIN Laptop 15.6 Inch, 4GB DDR4 128GB SSD Wind...",,4.5 out of 5 stars,"1,181 ratings",Not Available
4,"Acer Aspire 5 A515-56-347N Slim Laptop - 15.6""...",,4.3 out of 5 stars,"2,107 ratings",In Stock
5,"Lenovo 15.6"" Laptop, IdeaPad 1, 20GB RAM, 1TB ...",,4.4 out of 5 stars,17 ratings,Not Available
6,"Acer Aspire 5 15.6"" FHD IPS Slim Laptop 2023 N...",,4.3 out of 5 stars,223 ratings,Not Available
7,"HP 17.3"" Flagship HD+ Business Laptop, 16GB DD...",,4.3 out of 5 stars,58 ratings,Not Available
8,"HP Newest 15.6 FHD IPS Flagship Laptop, 11th G...",,4.3 out of 5 stars,198 ratings,Not Available
9,"HP 2021 Stream 14"" HD SVA Laptop Computer, Int...",,4.2 out of 5 stars,"1,098 ratings",Not Available
